In [1]:
import torch, open_clip, pathlib, gc
from onnxconverter_common import float16
from onnxruntime.quantization import quantize_dynamic, QuantType

In [2]:
MODEL = "ViT-H-14-quickgelu"
CKPT = "dfn5b"  # Hugging Face ID or local .pt
OPSET = 17
OUT_FP32 = pathlib.Path("clip_text_fp32.onnx")
OUT_FP16 = pathlib.Path("clip_text_fp16.onnx")
OUT_INT8 = pathlib.Path("clip_text_int8.onnx")

In [3]:
full, _, _ = open_clip.create_model_and_transforms(
    MODEL, pretrained=CKPT, force_custom_text=True, device="cpu"
)
text_enc = full.text.eval().cpu()
del full.visual
gc.collect()

tok = open_clip.get_tokenizer(MODEL)
dummy = tok(["dummy"]).to(torch.int64)

In [4]:
class Wrapper(torch.nn.Module):
    def __init__(self, enc):
        super().__init__()
        self.enc = enc

    def forward(self, input_ids):
        return self.enc(input_ids)

In [5]:
torch.onnx.export(
    Wrapper(text_enc),
    dummy,
    OUT_FP32.as_posix(),
    input_names=["input_ids"],
    output_names=["embeddings"],
    dynamic_axes={"input_ids": {0: "batch"}, "embeddings": {0: "batch"}},
    opset_version=OPSET,
    do_constant_folding=True,
)
print(f"📝  wrote {OUT_FP32.name}")

/home/gustofarbi/development/clip-onnx/.venv/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:5859: UserWarning: Exporting aten::index operator of advanced indexing in opset 17 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect results.
  warnings.warn(


📝  wrote clip_text_fp32.onnx


In [6]:
model_object = float16.convert_float_to_float16_model_path(OUT_FP32.as_posix(), keep_io_types=True)
print(f"📝  wrote {OUT_FP16.name}")

#torch.onnx.save(model_object, OUT_FP16.as_posi x())

/home/gustofarbi/development/clip-onnx/.venv/lib/python3.10/site-packages/onnxconverter_common/float16.py:53: UserWarning: the float32 number -6.335829993986408e-08 will be truncated to -1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(neg_max, -min_positive_val))
/home/gustofarbi/development/clip-onnx/.venv/lib/python3.10/site-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 5.081755904967622e-09 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
/home/gustofarbi/development/clip-onnx/.venv/lib/python3.10/site-packages/onnxconverter_common/float16.py:53: UserWarning: the float32 number -8.225588032928499e-08 will be truncated to -1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(neg_max, -min_positive_val))
/home/gustofarbi/development/clip-onnx/.venv/lib/python3.10/site-packages/onnxconverter_common/float16.py:43: UserWarning: t

📝  wrote clip_text_fp16.onnx


/home/gustofarbi/development/clip-onnx/.venv/lib/python3.10/site-packages/onnxconverter_common/float16.py:53: UserWarning: the float32 number -8.955023389489725e-09 will be truncated to -1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(neg_max, -min_positive_val))
/home/gustofarbi/development/clip-onnx/.venv/lib/python3.10/site-packages/onnxconverter_common/float16.py:50: UserWarning: the float32 number -inf will be truncated to -10000.0
  warnings.warn("the float32 number {} will be truncated to {}".format(neg_min, -max_finite_val))


In [7]:
model_object.save(OUT_FP16.as_posix())

AttributeError: export

In [6]:
quantize_dynamic(
    model_input=OUT_FP32.as_posix(),
    model_output=OUT_INT8.as_posix(),
    weight_type=QuantType.QInt8,
)
print(f"📝  wrote {OUT_INT8.name}")

📝  wrote clip_text_int8.onnx
